<div style='width:20em'><img src='img/logo-igm.png'></div>
<div style='font-size:larger'><strong>Algorithmique et programmation 2</strong><br>
L1 Mathématiques - L1 Informatique<br>
Semestre 2
</div>

# Algorithmes de recherche dans une liste

## Problème 1 : recherche dans une liste quelconque

### Énoncé du problème

Étant donnée une liste `lst` et un élément `x`, `lst` contient-elle `x` ?

- Problème extrêmement important
- Sous une forme ou une autre, résolu des millions de fois par jour
- En Python : `x in lst`

### Algorithme : recherche exhaustive (itérative)

- Pour chaque élément `elem` de `lst`
    - Si `elem` vaut `x`, répondre `True`
- À la fin du parcours, répondre `False`

#### Implémentation

In [ ]:
def rech_exh_iter(lst, x):
    for elem in lst:
        if elem == x:
            return True
    return False

In [ ]:
lst = list(range(10))
rech_exh_iter(lst, 9), rech_exh_iter(lst, 10)

#### Preuve de correction
    
Par récurrence : si $k$ est le nombre de tours de boucles déjà réalisé, `lst[:k]` ne contient pas `x`.
- Vrai quand aucune itération n'a été réalisée ($k = 0$).
- Supposons la propriété vraie après $k$ itérations.
    - Si `len(lst)` est $k$, la boucle s'arrête et le résultat est `False`. Par HR, `lst[:k] == lst` ne contient pas `x`.
    - Sinon, il existe un élément `lst[k]`. Si `lst[k] == x` le résultat est `True` et il n'y a pas d'autre itération. Sinon on a bien que `lst[:k+1]` ne contient pas `x`.

On en déduit que l'algorithme calcule le bon résultat.

#### Complexité

- En temps : $O(n)$ dans le pire cas, où $n$ est `len(lst)`.
    - Preuve : maximum $n$ itérations, chaque itération en $O(1)$.
- En espace : $O(1)$ espace supplémentaire
    - On ne compte pas la liste `lst` elle-même
    - On admet que la gestion de la boucle n'occupe qu'un espace constant

### Algorithme : recherche exhaustive (récursive)

- Si `lst` est vide, répondre `False`
- Sinon, si `lst[0]` vaut `x`, répondre `True`
- Sinon, rechercher `x` dans la liste privée de son premier élement

#### Implémentation (version naïve)

In [ ]:
def rech_exh_rec(lst, x):
    if len(lst) == 0:
        return False
    elif lst[0] == x:
        return True
    else:
        return rech_exh_rec(lst[1:], x)

In [ ]:
lst = list(range(10))
rech_exh_rec(lst, 9), rech_exh_rec(lst, 10)

#### Preuve de correction
    
Par récurrence sur la taille de la liste :
- Résultat correct sur une liste vide
- Supposons la propriété vraie après sur toute liste de taille au plus $k$, supposont `lst` de taille $k+1$
    - Si `lst[0] == x` le résultat est correct
    - Sinon, par HR l'appel récursif renvoie le bon résultat sur `lst[1:]`, et comme `lst[0] != x` c'est aussi le bon résultat sur `lst` toute entière

#### Complexité

- En temps : $O(n^2)$ dans le pire cas, où $n$ est `len(lst)`.
    - Preuve : complexité majorée par la suite $T(0) = c$, $T(n) = c.(n-1) + T(n-1)$
    - Suite déjà rencontrée en cours : $T(n) \in O(n^2)$
    - Coupable : calcul de `lst[1:]` !
- En espace : $O(n^2)$ dans le pire cas
    - Gestion de la pile : $O(1)$ par appel
    - Recopie de la liste à chaque appel !
    - Même calcul que ci-dessus

Rappel :

$$
\sum_{i=0}^{n-1} i = \big( 0 + 1 + \cdots + (n-2) + (n-1) \big) = \frac{n(n-1)}{2} \in O(n^2)
$$

#### Implémentation récursive (version améliorée)

In [ ]:
def rech_exh_rec_bis(lst, x, i=0):
    if len(lst) == i:
        return False
    elif lst[i] == x:
        return True
    else:
        return rech_exh_rec_bis(lst, x, i+1)

In [ ]:
lst = list(range(10))
rech_exh_rec_bis(lst, 9), rech_exh_rec_bis(lst, 10)

#### Preuve de correction
    
Par récurrence sur `len(lst) - i` : résultat correct sur `lst[i:]` 
- Propriété vraie si `len(lst) - i` nul : la fonction renvoie `False` et `lst[i:]` est bien vide
- Supposons la propriété vraie pour `len(lst) - i` au plus $k$, soit `len(lst) - i` $=k+1$
    - Si `lst[i] == x` le résultat est correct
    - Sinon, par HR l'appel récursif renvoie le bon résultat sur `lst[i+1:]`, et comme `lst[i] != x` c'est aussi le cas sur `lst[i:]`

#### Complexité

- En temps : $O(n)$ dans le pire cas, où $n$ est `len(lst)`.
    - Preuve : complexité majorée par la suite $T(0) = c$, $T(n) = c + T(n-1)$
- En espace : $O(n)$ dans le pire cas
    - Gestion de la pile : $O(1)$ par appel
    - Même calcul que ci-dessus

### Chronométrage



In [ ]:
import matplotlib.pyplot as plt

def trace_courbes(abscisses, series,
                  title='', xlabel='', ylabel=''):
    """Pour chaque couple (nom, ordonnées) de la liste series, trace la courbe
    des ordonnees avec la legende nom dans matplotlib. L'abscisse des points
    est donnée par la liste abscisses.
    Le paramètres optionnels title, xlabel et ylabel donnent le titre du
    graphique et les étiquettes des axes x et y."""
    for nom, donnees in series:
        plt.plot(abscisses, donnees, label=nom)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend()

In [ ]:
from timeit import timeit

def chrono(fonction, args, nombre=100000):
    """Renvoie le temps d'exécution en secondes de 'nombre' répétitions de
    l'appel fonction(*args)."""

    # le plus simple est de passer à la fonction timeit une fonction sans
    # argument, donc on doit définir une fonction auxiliaire
    def f():
        # passe un par un chaque argument de la liste args à
        # la fonction
        return fonction(*args)

    return timeit(f, number=nombre)

In [ ]:
def chrono_et_trace_rech(xs, fs=(rech_exh_iter, rech_exh_rec, rech_exh_rec_bis)):
    ys = []
    for f in fs:
        times = []
        for x in xs:
            lst = list(range(x))
            times.append([chrono(f, [lst, x], 100)])
        ys.append((f.__name__, times))
    trace_courbes(xs, ys)

In [ ]:
xs = list(range(0, 1001, 100))
chrono_et_trace_rech(xs, (rech_exh_iter, rech_exh_rec, rech_exh_rec_bis))

### Peut-on faire mieux ?

On peut démontrer qu'il faut faire au moins `len(lst)` accès à la liste pour répondre correctement  
*(borne inférieure de complexité sur le **problème**)*

- Soit `f` une fonction de recherche effectuant moins de `len(lst)` accès sur au moins une liste `lst`
- Soit `x` un élément n'appartenant pas à `lst`, `f(lst, x)` renvoie forcément `False`
- Soit `i` l'indice d'un élément non vu
    - Si l'on avait remplacé `lst[i]` par `x` la fonction aurait aussi renvoyé `False` (résultat incorrect)
  
**Conclusion :** si on ne sait rien sur la liste, on ne peut
pas faire mieux que `len(lst)` comparaisons !

## Problème 2 : recherche dans une liste triée

### Énoncé du problème

Étant donnée une liste `lst` d'éléments **comparables** deux à deux, **triée** dans l'ordre croissant, et un élément `x`, `lst` contient-elle `x` ?

Qu'est-ce qui change par rapport au problème précédent ?

### Préambule : jeu de la devinette

**Objectif :** deviner un nombre entier compris entre 1 et $N$, en posant le moins de questions possible. 

Trois personnes jouent :
- Devinante (**D**)
- Répondante (**R**)
- Arbitre (**A**)

On commence par prendre $N = 15$.

#### Déroulement :

1.  **R** choisit un entier $n$ entre 1 et $15$ et le
    communique secrètement à **A**.

2.  **D** propose à **R** un entier $a$ entre 1 et 15.

3.  **R** répond "égal" si $n = a$, "plus petit" si $n < a$, "plus grand" si
    $n > a$. **A** vérifie que la réponse est honnête.

4.  Si **D** a proposé $a = n$, la partie est terminée, **A** annonce le score. Sinon,
    on recommence au point 2.

But de **D** : deviner $n$ en faisant le moins possible de
propositions (en comptant la proposition finale, pour laquelle $a = n$). Le
nombre de propositions est appelé *score* de la partie. **D** souhaite donc
gagner avec le plus *petit* score possible.

#### Travail demandé

1. Faites des groupes de 3 et jouez quelques parties
2. Décrivez une stratégie pour **D** pour gagner
   *toutes* les parties avec un score le plus petit possible. 
3. Quel est le meilleur score que **D** est **sûre** d'atteindre (*score maximal pour $N = 15$*) ?
4. Quel est le meilleur score pour $N = 31$ ? Et pour $N = 50$ ? Et pour $N$
   quelconque ?

#### Bilan

- En jouant "bien", quand $N = 15$, **D** peut garantir un score maximal de 4. 
- Stratégie : si les nombres possibles sont les $g \leq a \leq d$, on propose $(g + d) / 2$ (arrondi comme on veut).
- Pourquoi ça marche : 15 possibilités au début, $\leq 7$ après 1 question, $\leq 3$ après 2 questions, $\leq 1$ après 3 questions, trouvé à la quatrième.
- Quand $N = 31$, 5 questions suffisent. Quand $N = 50$, 6 questions. Et pour $N$ quelconque... voir la suite.

### Retour aux listes

**Idée d'algorithme (ou *stratégie*) :** procéder par *dichotomie*

- Si on voit un élément trop petit, inutile de chercher avant
- Si on voit un élément trop grand, inutile de chercher après
- Pour éliminer le plus de cas possible on regarde toujours au
    milieu de l'intervalle

**Exemple :** on recherche l'élément `112`

![Recherche dichotomique](img/dicho-0.png)

`m = (g + d) // 2 = 9`

![Recherche dichotomique](img/dicho-1.png)

`lst[9] < 112` : on continue la recherche à droite de `m`

![Recherche dichotomique](img/dicho-2.png)

`g = m + 1 = 10`

![Recherche dichotomique](img/dicho-3.png)

`m = (g + d) // 2 = 14`

![Recherche dichotomique](img/dicho-4.png)

`lst[14] > 112` : on continue la recherche à gauche de `m`

![Recherche dichotomique](img/dicho-5.png)

`d = m - 1 = 13`

![Recherche dichotomique](img/dicho-6.png)

`m = (g + d) // 2 = 11`

![Recherche dichotomique](img/dicho-7.png)

`lst[11] < 112` : on continue la recherche à droite de `m`

![Recherche dichotomique](img/dicho-8.png)

`g = m + 1 = 12`

![Recherche dichotomique](img/dicho-9.png)

`m = (g + d) // 2 = 12`

![Recherche dichotomique](img/dicho-10.png)

`lst[12] < 112` : on continue la recherche à droite de `m`

![Recherche dichotomique](img/dicho-11.png)

`g = m + 1 = 13`

![Recherche dichotomique](img/dicho-12.png)

`m = (g + d) // 2 = 13`, et `lst[13] == 112`

![Recherche dichotomique](img/dicho-13.png)

### Algorithme : recherche par dichotomie (itérative)

Soient `lst` une liste croissante de `n` éléments, `x` un élément

- Posons `g = 0`, `d = n - 1`
- Tant que l'intervalle $[g, d]$ contient au moins un élément :
    - Calculer `m = (g + d) // 2`
    - Si `lst[m] == x`, répondre `True`
    - Si `lst[m] < x`, fixer `g` à `m+1`
    - Si `lst[m] > x`, fixer `d` à `m-1`
- Si l'on sort de la boucle, répondre `False`

#### Implémentation

In [ ]:
def dicho_iter(lst, x):
    g, d = 0, len(lst) - 1
    while g <= d:
        m = (g + d) // 2
        if lst[m] == x:
            return True
        elif lst[m] < x:
            g = m + 1
        else:
            d = m - 1
    return False

In [ ]:
lst = list(range(10))
dicho_iter(lst, 3), dicho_iter(lst, 3.5)

#### Preuve de correction
    
Par récurrence sur le nombre de tours de boucle : en début d'itération, ni `lst[:g]` ni `lst[d+1:]` ne contiennent `x`
- Propriété vraie après 0 itérations : `lst[:g]` et `lst[d+1:]` sont vides
- Supposons la propriété vraie après `k` itérations. Si `lst[m] == x` la fonction renvoie un résultat correct et termine. Sinon :
    - Si `lst[m] < x`, au début de l'itération suivante `g` vaut `m+1`. Or comme `lst` est croissante, tous les éléments d'indice au plus `g-1` sont inférieurs à `x` (et donc différents)
    - Si `lst[m] > x`, au début de l'itération suivante `d` vaut `m-1`. Or comme `lst` est croissante, tous les éléments d'indice au moins `d+1` sont supérieurs à `x` (et donc différents)

#### Preuve de terminaison

- La quantité `d - g` diminue strictement à chaque tour de boucle
- La boucle s'arrête si `d - g` devient négatif ou nul
- Donc le nombre de tours ne peut pas être infini

#### Complexité

- Chaque tour de boucle prend un temps $O(1)$
- Au début du premier tour de boucle, on cherche parmi `len(lst)` éléments (`d - g` vaut `len(lst) - 1`)
- À chaque tour, on élimine une possibilité plus la moitié de ce qui reste (`d - g` décrémenté et divisé par deux)
- Si `d - g <= 1` la boucle s'arrête à la fin du tour

Parmi combien de nombres peut-on trouver à coup sûr en $k$ tours de boucle ?

\begin{array}[c]{l|c|c|c|c|c|c|c}
  \hline
  k & 1 & 2 & 3 & 4 & 5 & 6 & \ldots \\\hline
  n & 1 & 3 & 7 & 15 & 31 & 63 & \ldots\\
  \hline
\end{array}

*Conjecture :* en $k$ tours on peut distinguer parmi $2^k - 1$ nombres

- Preuve : par récurrence
    - En 0 tours on peut distinguer parmi $ 2^0 - 1 = 0$ nombres
    - Si on sait chercher parmi $2^k - 1$ en $k$ questions, on peut chercher parmi $2 (2^k - 1) + 1 = 2^{k+1} - 1$ nombres en $k+1$ questions
- Suite déjà rencontrée dans le cours sur les tours de Hanoï

*Conclusion :* Pour $n$ quelconque, on devra donc chercher au plus en $k$ tours où $k$ est
  l'unique entier tel que :
  
\begin{align*}
 & & 2^{k-1} - 1 & < n \leq 2^k - 1 \\
\text{soit } & & 2^{k-1} & \leq n < 2^k \\
\text{soit } & & k-1 & \leq \log_2 n < k \\
\text{soit } & & k = & \lfloor \log_2 n \rfloor + 1
\end{align*}


On obtient donc une complexité en
$O(\log_2(n))$ au pire

![Nombre de comparaisons](img/graphe-comparaisons.png)

### Algorithme : recherche par dichotomie (récursive)

Soient `lst` une liste croissante de `n` éléments, `x` un élément

- Si la liste est vide, la réponse est `False`
- Sinon :
    - Calculer `m = len(lst) // 2`
    - Si `lst[m] == x`, répondre `True`
    - Si `lst[m] < x`, chercher dans `lst[m+1:]`
    - Si `lst[m] > x`, chercher dans `lst[:m]`

#### Implémentation récursive (naïve)

In [ ]:
def dicho_rec(lst, x):
    if len(lst) == 0:
        return False
    m = len(lst) // 2
    if lst[m] == x:
        return True
    elif lst[m] < x:
        return dicho_rec(lst[m+1:], x)
    else:
        return dicho_rec(lst[:m], x)

In [ ]:
lst = list(range(10))
dicho_rec(lst, 3), dicho_rec(lst, 3.5)

#### Variante : avec des indices

Soient `lst` une liste croissante de `n` éléments, `x` un élément, `g` et `d` deux indices

- Si `g > d`, la réponse est `False`
- Sinon :
    - Calculer `m = (g + d) // 2`
    - Si `lst[m] == x`, répondre `True`
    - Si `lst[m] < x`, chercher entre `m+1` et `d`
    - Si `lst[m] > x`, chercher entre `g` et `m-1`

#### Implémentation récursive (efficace)

In [ ]:
def dicho_rec_bis(lst, x, g=0, d=None):
    if d is None:
        d = len(lst) - 1
    if g > d:
        return False
    m = (g + d) // 2
    if lst[m] == x:
        return True
    if lst[m] > x:
        return dicho_rec_bis(lst, x, g, m-1)
    else:
        return dicho_rec_bis(lst, x, m+1, d)

In [ ]:
lst = list(range(10))
dicho_rec_bis(lst, 3), dicho_rec_bis(lst, 3.5)

#### Correction, terminaison, complexité de la version récursive

TODO ! 😅

### Chronométrage



In [ ]:
xs = list(range(0, 10000001, 1000000))
#chrono_et_trace_rech(xs, (rech_exh_iter, dicho_iter, dicho_rec, dicho_rec_bis))
chrono_et_trace_rech(xs, (dicho_iter, dicho_rec_bis))

### Peut-on faire mieux ?

#### Retour sur le jeu de la devinette

1.  **R** fait **semblant** de choisir un entier $n$ entre 1 et $15$.

2.  **D** propose à **R** un entier $a$ entre 1 et 15.

3.  **R** répond ce qu'elle veut. **A** vérifie que la réponse est cohérente avec les réponses précédentes.

4.  Si **R** a répondu « égal », la partie est terminée, **A** annonce le score. Sinon,
    on recommence au point 2.

But de **D** : deviner $n$ en faisant le moins possible de
propositions.  
But de **R** : forcer **D** à poser le plus de questions possible.

#### Travail demandé

1. Faites des groupes de 3 et jouez quelques parties
2. Décrivez une stratégie pour **R** pour forcer **D** à poser le plus de questions possible.
3. Quel est le meilleur score que **D** peut atteindre (*score minimal pour $N = 15$*) ?
4. Quel est le meilleur score pour $N = 31$ ? Et pour $N = 50$ ? Et pour $N$
   quelconque ?

#### Bilan

- En jouant "bien", quand $N = 15$, **R** peut forcer un score minimal de 4. 
- Stratégie : toujours répondre de manière à conserver le plus grand nombre de possibilités.
- Pourquoi ça marche : 15 possibilités au début, $\geq 7$ après 1 question, $\geq 3$ après 2 questions, $\geq 1$ après 3 questions, trouvé à la quatrième.
- Quand $N = 31$, 5 questions au minimum. Quand $N = 50$, 6 questions. Et pour $N$ quelconque... ?

## Bilan final

- La recherche d'élément dans une liste est un problème fondamental. 
- Quand la liste n'est pas triée, il n'existe pas de meilleur algorithme que la recherche exhaustive, en $O(n)$
- Quand la liste est triée, il existe un algorithme très efficace : la recherche par dichotomie, en $O(\log n)$
    - On peut démontrer qu'il est impossible de faire mieux ! (argument du jeu de la devinette, phase 2)
    - Plus d'informations en M1 info !